### CONFIGURATION

In [2]:
# GENERAL PYTHON IMPORTS
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)  #necessary for pileup files
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
# HARTWIG imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import analysePileup as aPILEUP
import venn as vn

In [30]:
# BED FILE
BED_PATH = "/Users/peterpriestley/hmf/repos/pipeline/settings/slicing/"
BED_FILE_NAME = "CPCT_Slicing.bed"

# COMBINED VCF CONFIG
VCF_SAMPLE = "CPCT02080037"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/highSomaticVariants/"
VCF_FILE_NAME = VCF_SAMPLE + "R_" + VCF_SAMPLE + "T_merged_somatics.vcf"
#VCF_FILE_NAME = "CPCT02140003R_CPCT02140003T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf"
SAMPLE_NAMES = {VCF_SAMPLE + 'T.mutect':'mutect', \
                VCF_SAMPLE + 'T.freebayes':'freebayes', \
                'TUMOR.strelka':'strelka', \
                'TUMOR.varscan':'varscan'}
#SAMPLE_NAMES = {'CPCT02140003T':'melted'}
VCF_SAMPLE2 = "CPCT02070011"
VCF_FILE_NAME2 = VCF_SAMPLE2 + "R_" + VCF_SAMPLE2 + "T_merged_somatics.vcf"
SAMPLE_NAMES2 = {VCF_SAMPLE2 + 'T.mutect':'mutect', \
                VCF_SAMPLE2 + 'T.freebayes':'freebayes', \
                'TUMOR.strelka':'strelka', \
                'TUMOR.varscan':'varscan'}
# PILEUP CONFIG
PILEUP_PATH = "/Users/peterpriestley/hmf/slice" + VCF_SAMPLE + "/"
NORMAL_PILEUP_FILENAME = VCF_SAMPLE + ".pileup"
TUMOR_PILEUP_FILENAME = VCF_SAMPLE + "T.pileup"

#MELT PARAMS
VCF_FILE_NAME = VCF_SAMPLE + "R_" + VCF_SAMPLE + "T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf"
SAMPLE_NAMES = {VCF_SAMPLE+'T':'melted'}

<h3> Load VCFs and Prepare DF

In [31]:
## LOAD BED IF REQUIRED
#bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)

In [32]:
# LOAD PROD VCF AND FILTER
#sampleBed = copy.deepcopy(bed)
df = aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,False)#True,sampleBed)

reading vcf file: CPCT02080037R_CPCT02080037T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
reading VCF File line: 1100000
reading VCF File line: 1200000
reading VCF File line: 1300000
Number variants loaded: 1343889


In [33]:
df['dbSNP'] = (df['id'].str.contains('rs'))
df['COSMIC'] = (df['id'].str.contains('COSM'))

### CONTAMINATION ANALYSIS

In [34]:
#df2 = aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME2,SAMPLE_NAMES2,VCF_SAMPLE2,True,False)

In [35]:
#df = df.set_index(['chrom','pos'])
#df2 = df2.set_index(['chrom','pos'])
#dfComb = pd.merge(df,df2, left_index=True,right_index=True,how='inner',suffixes=('', '_2'))

In [36]:
#len(df)

In [37]:
#len(df2)

In [38]:
#len(dfComb)

<h3> ANALYSIS

In [39]:
#INTERNAL 'PRECISION' + 'SENSITIVITY' per caller
outputdata = []
variantTypes = df.variantType.unique()
patientNames = df.patientName.unique()
for patientName in patientNames:
    for variantType in variantTypes:
        truthSet = len(df[(df['numCallers'] >= 1)&(df['variantType'] == variantType)&(df['patientName'] == patientName)])
        for columnName in list(df):
            if columnName.endswith('allele'):
                myCaller = columnName[:-6]
                truePositives = len(df[(df[myCaller+'allele'] != '')&(df['numCallers'] >=2) &\
                                   (df['variantType'] == variantType)&(df['patientName'] == patientName)])
                falseNegatives = len(df[(df[myCaller+'allele'] == '')&(df['numCallers'] >= 2) &\
                                    (df['variantType'] == variantType)&(df['patientName'] == patientName)])
                positives = len(df[(df[myCaller+'allele'] != '')&\
                                   (df['variantType'] == variantType)&(df['patientName'] == patientName)]) 
                if positives > 0:
                    outputdata.append([variantType, myCaller, truthSet,truePositives,\
                                   positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(truthSet),4)])
   
outputDF = pd.DataFrame(outputdata)
outputDF.columns = (['variantType','caller','truth','TP','FP','FN',\
                     'intPrecision','intSensitivity'])
outputDF.sort_values(['variantType','caller'])

,variantType,caller,truth,TP,FP,FN,intPrecision,intSensitivity
1,INDEL,melted,87412,0,87412,0,0.0,0.0
0,SNP,melted,1256477,0,1256477,0,0.0,0.0


In [43]:
dftemp = df[(df['variantType']!= "ASNP")].reset_index()
pd.pivot_table(dftemp, values='pos', index=['variantType'], columns=['dbSNP'], aggfunc='count')

dbSNP,False,True
variantType,,
INDEL,75626,11786
SNP,91932,1164545


<h3> SNP

In [ ]:
snpdf = df[(df.variantType == 'SNP')]

In [ ]:
#vn.venn([snpdf[snpdf.mutectallele != '']['chromPos'], \
#         snpdf[snpdf.strelkaallele != '']['chromPos'], \
#        snpdf[snpdf.freebayesallele != '']['chromPos'], \
#        snpdf[snpdf.varscanallele != '']['chromPos'] \
#        ],['mutect','strelka','freebayes','varscan'],figsize=(6,6))

### <hd3> Indel Venn

In [ ]:
#indeldf= df[(df.variantType == 'INDEL')]

In [ ]:
#vn.venn([indeldf[indeldf.strelkaallele != '']['chromPos'], \
#        indeldf[indeldf.freebayesallele != '']['chromPos'], \
#        indeldf[indeldf.varscanallele != '']['chromPos'] \
#        ],['strelka','freebayes','varscan'],figsize=(6,6))

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(snpdf):
    if columnName.endswith('allele'):
        ser = snpdf[(snpdf[columnName] != '') &(snpdf[columnName[:-6] + 'AF'] > 0.0)&(snpdf.numCallers > 0)][columnName[:-6] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=VCF_SAMPLE+" Allelic Frequency by Caller (AF > 0.0)",figsize=[10,6],ylim=[0,1])

### Allelic Depth Scatter Plot

In [ ]:
#AD vs DP single caller
caller1 = 'mutect'
tempdf = df[(df[caller1+'allele']!='')&(df[caller2+'allele']!='')]
tempdf.head()
tempdf.plot.scatter(caller1+'AF',caller1+'DP' \
                        ,figsize=[6,6] \
                        ,title="Comparison of "+caller1+" and "+caller2+" Allelic Frequency" \
                        ,xlim=[0,0.2],ylim=[0,250])

In [ ]:
# Cross caller
caller1 = 'mutect'
caller2 = 'strelka'
tempdf = df[(df[caller1+'allele']!='')&(df[caller2+'allele']!='')]
tempdf.head()
tempdf.plot.scatter(caller1+'AF',caller2+'AF' \
                        ,figsize=[6,6] \
                        ,title="Comparison of "+caller1+" and "+caller2+" Allelic Frequency" \
                        ,xlim=[0,1],ylim=[0,1])

### Allelic Depth Scatter Plot

In [ ]:
## RAINBOW
caller = 'strelka'
plt.scatter(df[(df[caller+'allele']!='')&(df['chrom'] == '17')]['chromFrac'], \
            df[(df[caller+'allele']!='')&(df['chrom'] == '17')][caller+'AF'], \
            s=10, c='b', marker="s")